In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from torch.autograd import Variable

"""Change to the data folder"""
train_path = "./new_train/"
val_path = "./new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=train_path)
val_dataset =  ArgoverseDataset(data_path=val_path)

In [16]:
batch_sz = 32

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp =  torch.FloatTensor(inp)
    out =  torch.FloatTensor(out)
    
    return [inp, out]

In [17]:
batch_sz = 32

def my_collate2(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    a_id =[numpy.dstack([scene['agent_id']]) for scene in batch]
    t_id =[numpy.dstack([scene['track_id']]) for scene in batch]
    s_id =[numpy.dstack([scene['scene_idx']]) for scene in batch] 
    #inp = torch.LongTensor(inp)
    #out = torch.LongTensor(out)
    inp =  torch.FloatTensor(inp)
    #out =  torch.FloatTensor(out)
    
    return [inp,a_id,t_id,s_id]

In [18]:
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=1)
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate2, num_workers=1)

In [19]:
from torch import nn,optim
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel,self).__init__()
        self.hidden_dim = 2048
        self.num_layers = 3
        self.lstm = nn.LSTM(240,self.hidden_dim, num_layers= self.num_layers, batch_first = True)
        self.lstm2 = nn.LSTM(self.hidden_dim,self.hidden_dim, num_layers= self.num_layers, batch_first = True)
        self.fc2 = nn.Linear(self.hidden_dim,240)

    def forward(self,x):
       #input : batch sze x  timesteps x 240
        #print("Input Dims: {}".format(x.shape))
        x,(h,c) = self.lstm(x)
        
        #print("Output Dims Encoder LSTM: {}".format(x.shape))
        
        #
        x,(h,c) = self.lstm2(x,(h,c))
        
        #print("Output Dims Decoder LSTM: {}".format(x.shape))
        
        #output = torch.ones(30,2)  
        #output[0] = x
        
        #for i in range(1, 30):
         #   output[i],(h,c) = self.lstm2(output[i-1])
          #  output[i] = self.fc2(self.lstm2(output[i-1]))

        #x,(h,c) = self.lstm2(x)

        #final hidden layer: batch_size x timesteps x hidden_dim
        #x = x .transpose(1,2)
        
        #print("After Transpose : {}".format(x.shape))
         
        #batch size x hidden_dim x timesteps
       # x = self.linear(x)
        x = self.fc2(x)
        #print("Output Dims Fully Connected : {}".format(x.shape))
        
        # last step to : project hidden_dim -> 240
        #x = x.transpose(1,2)

        return x

    def forward_test(self, x, num_steps = 30):
        res = []

        #Hidden states initiallized to 0 
        h = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        c = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        
        # run input for first 19 steps, take hidden state at last time step
        # perform projection an add to list of outputs
        # want to use the output of the 19 time step to predict the output of the 20th
        for step in range((num_steps)):
            # output, [hidden_states]
            x,(h,c) = self.lstm(x,(h,c))
            #take the hidden state at the last time step 
            x = x[:,-1:]
            x = x.tranpose(1,2)
            x = self.linear(x)
            x = x.transpose(1,2)
            res.append(x)
        res = torch.cat(res,1)
        return res

In [20]:
# Throw all the input and model information into the gpu 
model = LSTMModel().cuda()   # will turn float tensors into cuda tensors

optimizer = optim.Adam(model.parameters(), lr= 1e-3)

# Exponentially Moving Average 
loss_ema = -1
loss_ema2 = -1

In [21]:
for epoch in range(10):
    # after a few epochs if nothing is improving lower your learning rate 
    for i_batch, sample_batch in enumerate(train_loader):
        
        model.zero_grad()
        optimizer.zero_grad()
        
        inp,out = sample_batch
        #print("Inp Size :{}".format(inp.shape))
        #print("Out Size :{}".format(out.shape))
        inp = inp.cuda()
        out = out.cuda()
        
        #print("Output Labels Dimensions: {}".format(out.shape))

        # COncatenate input and output along the timestep axis : batch_sz x 60 x 49 x 4
        mixed = torch.cat([inp,out],2).transpose(1,2).reshape((-1,49,240))#.float()
        
        #print("Mixed Size: {}".format(mixed.shape))

       # Feed every time step but the final time step into the model 
        
        #print("19 + 30 time steps:{}".format(mixed[:,:,-1].shape))
        #print("30 Time Steps Output: {}".format(mixed[:,:,-1][:,-30:].shape))
        
        time_step = mixed[:,:,:][:,-30:,:]
        #print("time step dims: {}".format(time_step.shape))
        
        #y_pred= model(mixed[:,:,-1])[:,-30:] # in: first 48 time steps , gives predictions timesteps 2-49
        
       
        
        
        y_pred = model(time_step)

        
        # 32x30x240 => 32x60x30x4  comparing to ground_truth
        y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
        #print("Predicted Y : {}".format(y_pred.shape))

        # compare prediction to actual output [Ground Truth]. Take the root mean squared error
        loss = (torch.mean((y_pred - out)**2))**0.5
       
        loss.backward()
        optimizer.step()
        
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss*0.01

        # Measure the difference between using the ground truth value and not using it[ Used for validation data ]
        #with torch.no_grad():
         #   y_pred2 = model.forward_test(inp.transpose(1,2). reshape((-1,19,240)))
          #  y_pred2 = y_pred2.reshape((-1,30,60,4)).transpose(1,2)
           # loss2 = torch.mean((y_pred2 - out)**2)**0.5

        
        # Every 10th iteration print out the loss and exponentially moving average 
        if i_batch %10 == 0:
            print("loss full", epoch, i_batch, loss_ema.item(),loss.item())
            #print("loss test", epoch, i_batch, loss_ema2.item(),loss2.item())
            
        #if i_batch > 3215:
         #   break
            


loss full 0 0 589.5530395507812 589.5530395507812
loss full 0 10 590.9843139648438 685.9768676757812
loss full 0 20 588.4910888671875 549.43798828125
loss full 0 30 587.2839965820312 552.4968872070312
loss full 0 40 584.171142578125 533.3008422851562
loss full 0 50 582.1233520507812 546.3180541992188
loss full 0 60 579.6312255859375 599.8087768554688
loss full 0 70 575.322021484375 564.2852172851562
loss full 0 80 571.890380859375 561.4942016601562
loss full 0 90 570.5662841796875 580.6803588867188
loss full 0 100 565.5914916992188 465.9576110839844
loss full 0 110 564.594482421875 578.7413330078125
loss full 0 120 561.4268798828125 547.8177490234375
loss full 0 130 558.1302490234375 568.7534790039062
loss full 0 140 556.0145874023438 517.4682006835938
loss full 0 150 553.1687622070312 566.736083984375
loss full 0 160 552.82421875 509.1949768066406
loss full 0 170 550.6397705078125 525.4526977539062
loss full 0 180 546.8981323242188 461.0736083984375
loss full 0 190 542.8639526367188 4

In [22]:
print(y_pred.shape)
print(out.shape)

torch.Size([22, 60, 30, 4])
torch.Size([22, 60, 30, 4])


In [23]:
#Find the index of an agent_id within the track_id of a current scence 
def find_index(track_id, agent_id):
    track_id = list(track_id)
    index = track_id.index(agent_id)
    return index

In [24]:
#   for epoch in range(20):

# y = ["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "v29", "v30", "v31", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47", "v48", "v49", "v50", "v51", "v52", "v53", "v54", "v55", "v56", "v57", "v58", "v59", "v60"]
final_result = []
for i_batch , sample_batch in enumerate(val_loader):
    inp,a_id,t_id,s_id = sample_batch
    
    s_id = list(numpy.concatenate(s_id).flat)
    a_id = list(numpy.concatenate(a_id).flat)
    t_id = numpy.array(t_id)
    
    index_list_per_batch = []
    
    #Generate the indexes we need for each agent row 
    for i in range(32):
        flat_t_id = numpy.array(t_id[i])
        flat_t_id = flat_t_id[:,0,0]
        current_agent_id = a_id[i]
        
        
        index = find_index(flat_t_id, current_agent_id)
        
        index_list_per_batch.append(index)
    

    inp = inp.cuda()
    out =  torch.zeros(32,60,30,4)
    out = out.cuda()
    #print(inp.shape)
    #print(out.shape)
    mixed = torch.cat([inp,out],2).transpose(1,2).reshape((-1,49,240))
    #Use the model to ouput predicted positions for each agent in the 32 scenes
    #print("Inp: {}".format(inp.shape))
    #inp =  inp.transpose(1,2).reshape((-1,19,240))
   
   
    
    # Flatten tensor to the timestep dimension
    time_step = mixed[:,:,:][:,-30:,:]
    #time_step = inp[:,:,:][:,:,:]
    #print("Time Step Dim: {}".format(time_step.shape))
    
   
    
    y_pred = model(time_step)
    y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
    #y_pred = model(time_step).reshape((-1,30,60,4))

    #print("Model Output : {}".format(y_pred.shape))
    
    #y_pred = y_pred
    #print("Reshape and Transpose {}".format(y_pred.shape))
    
    
    
    #y_pred = model(inp.reshape((len(inp),-1))).reshape((-1,60,30,4))
    
    
    # Retrive only the positions 
    y_pred = y_pred[:,:,:,[0,1]]
    
    #print("Only Positions : {}".format(y_pred.shape))
    for i in range(32):
        row_result = y_pred[i,index_list_per_batch[i],:,:]
        
        
        row_result = torch.flatten(row_result)
         
        
        
        row_result = row_result.cpu()
        row_result = row_result.detach().numpy()
        row_result = numpy.hstack(([s_id[i]], row_result))
        final_result.append(row_result)
        
        
        
    
    
    
    
    
    
   
     

In [25]:
#import pandas as pd 
#print(final_result)
#y = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "v29", "v30", "v31", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47", "v48", "v49", "v50", "v51", "v52", "v53", "v54", "v55", "v56", "v57", "v58", "v59", "v60"]
#y = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60']
#final_result.insert(0, numpy.array(y))
final = numpy.array(final_result)
# var = numpy.stack((y, final))
print(final)
#final_df = pd.DataFrame(final,index=None,columns=y)
#final_df.columns = y 
#final_df
#final_df.to_csv("submission.csv")
#final_df = pd.DataFrame(final).to_csv("submission.csv", header=None, index=None)
#final_df.columns = y
numpy.savetxt("submission.csv", final, delimiter=",",fmt='%s')


[[10002.           333.55883789   219.62319946 ...   -98.21150208
    -59.73562622   -69.53401947]
 [10015.           383.61877441   160.2822876  ...   112.07587433
    114.87808228    95.94880676]
 [10019.           333.55883789   219.62319946 ...   -98.21150208
    -59.73562622   -69.53401947]
 ...
 [ 9905.           333.55883789   219.62319946 ...   -98.21150208
    -59.73562622   -69.53401947]
 [ 9910.          1076.77246094   846.25177002 ...   526.78149414
    610.15209961   560.97149658]
 [ 9918.           422.30133057   128.66337585 ...   181.54083252
    286.46658325   193.46495056]]
